# Proyecto Final IH: Recomendador de Codewars|Github

In [ ]:
# https://www.codewars.com/users/leaderboard

In [ ]:
# imports 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

## Fase 1: Obtención de usuarios de forma iterativa

### Leaderboard
Top 500 de codewars

In [ ]:
# URL

url = 'https://www.codewars.com/users/leaderboard'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

In [2]:
# usuarios de leaderboard
def get_top_500(url='https://www.codewars.com/users/leaderboard'): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text for e in soup.select('tr a')}

# users = '\n'.join(get_top_500())
# set_top500
# users
# get_top_500()

In [3]:
# Salvamos los nombres en un documento
def save_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Save set/list of users to file '''
    with open(filename, 'w') as f: 
        f.write('\n'.join(users)) 
# save_set_users({'hola', 'adios'})

In [4]:
def save_string_users(users, filename='../output/codewars-users.txt'): 
    ''' Save string of users to file '''
    with open(filename, 'w') as f: 
        f.write(users) 
# save_set_users(users)

In [5]:
def add_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Append to file '''
    with open(filename, 'a') as f: 
        f.write(users) 
# add_set_users(users)

In [6]:
def load_set_users(filename='../output/codewars-users.txt'): 
    ''' Read from file '''
    with open(filename, 'r') as f: 
        users = f.readlines()
    return {user.strip() for user in users}
# len(load_set_users())

### Get Social

In [9]:
# get users in clan

# https://www.codewars.com/users/albertogcmr/following
# https://www.codewars.com/users/albertogcmr/followers
# https://www.codewars.com/users/albertogcmr/allies

def get_social(user): 
    res = set()
    for link in ['following', 'followers', 'allies']: 
        url = 'https://www.codewars.com/users/{}/{}'.format(user, link)
        html = requests.get(url).content

        soup = BeautifulSoup(html, "lxml")
        res = res.union({e.text for e in soup.select('table a')})
    return res

# get_social('albertogcmr')

### Get stats from user

In [10]:
def get_stats(user): 
    url = 'https://www.codewars.com/users/{}'.format(user)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    x = {}
    for s in soup.select('.stat-box div'): 
        if s.text.split(':')[0] != 'Profiles': 
            x[s.text.split(':')[0]] = s.text.split(':')[1]
        else: 
            try: 
                for e in s.find_all('a', href=True): 
                    if 'github' in e['href']: 
                        x['github'] = e['href']
                    if 'linkedin' in e['href']: 
                        x['linkedin'] = e['href']
            except: 
                x['github'] = ''
                x['linkedin'] = ''
    return x

# get_stats('boyander')

### Iteración para apliar número de usuarios

In [11]:
top500 = get_top_500()
total = set().union(top500)
users = total

In [13]:
print(len(total))

for i, user in enumerate(users): 
    total = total.union(get_social(user))
    print(i, '\t', len(total))

print(len(total))

499
499
500
514
526
531
553
557
563
579
587
603
613
627
656
670
670
677
685
704
705
712
712
725
733
748
761
776
781
781
784
796
796
799
818
819
822
832
836
852
858
858
873
881
893
894
896
913
918
918
931
931
936
940
957
960
974
983
985
988
993
996
996
996
1003
1014
1032
1033
1052
1052
1060
1060
1072
1080
1084
1086
1104
1122
1135
1135
1151
1166
1168
1198
1208
1208
1227
1233
1233
1234
1246
1250
1258
1273
1273
1273
1275
1293
1293
1294
1300
1300
1310
1310
1311
1313
1313
1328
1339
1342
1357
1374
1377
1391
1399
1408
1419
1419
1439
1439
1450
1462
1486
1492
1503
1505
1521
1543
1547
1564
1568
1583
1590
1595
1595
1609
1609
1617
1631
1639
1639
1653
1657
1667
1677
1677
1691
1701
1716
1723
1726
1740
1745
1748
1751
1751
1771
1771
1784
1807
1817
1822
1827
1847
1847
1852
1872
1882
1883
1900
1903
1904
1910
1916
1919
1921
1932
1949
1949
1949
1955
1958
1976
1978
1983
1994
2000
2000
2000
2003
2009
2010
2019
2028
2028
2028
2030
2045
2047
2059
2064
2079
2083
2084
2104
2116
2116
2128
2150
2166
2166
2166
2174

In [27]:
# save_set_users(users=total, filename='../output/no-borrar-usuarioscodewars-2.txt')
total = load_set_users('../output/no-borrar-usuarioscodewars-2.txt')

In [28]:
len(total)

3924

In [32]:
total = total.union({'albertogcmr'})
len(total)

3925

In [35]:
for user in get_social('albertogcmr'): 
    total = total.union(get_social(user))
    print(len(total))

print(len(total))

3944
3945
3945
3945
3945
3945
3945
3945
3945
3945
3945
3949
3949
3949
3949
3953
3953
3953


In [42]:
len(total)

3953

## Fase 2: Obtención de datos de usuario

In [80]:
def get_user_api(user='albertogcmr'): 
    url = 'https://www.codewars.com/api/v1/users/{}'.format(user)
    response = requests.get(url)
    return response.json()
alberto = get_user_api()
alberto

{'username': 'albertogcmr',
 'name': 'Alberto García cobo',
 'honor': 727,
 'clan': 'Ironhack',
 'leaderboardPosition': 12890,
 'skills': ['java',
  'c',
  'python',
  'jquery',
  'javascript',
  'crm',
  'erp',
  'itil'],
 'ranks': {'overall': {'rank': -4,
   'name': '4 kyu',
   'color': 'blue',
   'score': 888},
  'languages': {'python': {'rank': -4,
    'name': '4 kyu',
    'color': 'blue',
    'score': 857},
   'sql': {'rank': -7, 'name': '7 kyu', 'color': 'white', 'score': 24},
   'c': {'rank': -8, 'name': '8 kyu', 'color': 'white', 'score': 10}}},
 'codeChallenges': {'totalAuthored': 0, 'totalCompleted': 89}}

In [50]:
users_10 = [u for u in load_set_users('../output/no-borrar-usuarioscodewars-2.txt')][:10]

In [54]:
users_10

['c.e.buco',
 'Tciitb',
 'Andrew_8',
 'mi5hu',
 'abhishek0318',
 'ken11zer01',
 'hardentoo',
 'delfuego',
 'megawatt',
 'Dront']

In [55]:
users_10_jsons = []
for user in users_10: 
    users_10_jsons.append(get_user_api(user))
    
users_10_jsons 

[{'username': 'c.e.buco',
  'name': None,
  'honor': 11,
  'clan': 'The Software Guild',
  'leaderboardPosition': 340049,
  'skills': None,
  'ranks': {'overall': {'rank': -8,
    'name': '8 kyu',
    'color': 'white',
    'score': 12},
   'languages': {'java': {'rank': -8,
     'name': '8 kyu',
     'color': 'white',
     'score': 12}}},
  'codeChallenges': {'totalAuthored': 0, 'totalCompleted': 5}},
 {'username': 'Tciitb',
  'name': None,
  'honor': 16,
  'clan': 'IIT BHU',
  'leaderboardPosition': 298013,
  'skills': None,
  'ranks': {'overall': {'rank': -7,
    'name': '7 kyu',
    'color': 'white',
    'score': 24},
   'languages': {'python': {'rank': -7,
     'name': '7 kyu',
     'color': 'white',
     'score': 24}}},
  'codeChallenges': {'totalAuthored': 0, 'totalCompleted': 5}},
 {'username': 'Andrew_8',
  'name': None,
  'honor': 501,
  'clan': 'MITx',
  'leaderboardPosition': 22996,
  'skills': None,
  'ranks': {'overall': {'rank': -5,
    'name': '5 kyu',
    'color': 'yell

In [56]:
data = pd.DataFrame(users_10_jsons)
data.head(10)

,clan,codeChallenges,honor,leaderboardPosition,name,ranks,skills,username
0,The Software Guild,"{'totalAuthored': 0, 'totalCompleted': 5}",11,340049,None,"{'overall': {'rank': -8, 'name': '8 kyu', 'col...",None,c.e.buco
1,IIT BHU,"{'totalAuthored': 0, 'totalCompleted': 5}",16,298013,None,"{'overall': {'rank': -7, 'name': '7 kyu', 'col...",None,Tciitb
2,MITx,"{'totalAuthored': 0, 'totalCompleted': 58}",501,22996,None,"{'overall': {'rank': -5, 'name': '5 kyu', 'col...",None,Andrew_8
3,GTS RO,"{'totalAuthored': 0, 'totalCompleted': 49}",297,48226,,"{'overall': {'rank': -5, 'name': '5 kyu', 'col...",[],mi5hu
4,IIT BHU,"{'totalAuthored': 0, 'totalCompleted': 21}",188,74276,None,"{'overall': {'rank': -6, 'name': '6 kyu', 'col...",None,abhishek0318
5,Central Philippine University,"{'totalAuthored': 0, 'totalCompleted': 39}",208,67793,Kenneth Ken L. Tan,"{'overall': {'rank': -6, 'name': '6 kyu', 'col...",[],ken11zer01
6,None,"{'totalAuthored': 1, 'totalCompleted': 115}",1206,5662,Sergey,"{'overall': {'rank': -4, 'name': '4 kyu', 'col...",[],hardentoo
7,Government Drones,"{'totalAuthored': 1, 'totalCompleted': 251}",5585,445,Jason Levine,"{'overall': {'rank': -2, 'name': '2 kyu', 'col...","[java, javascript, html, css, sql]",delfuego
8,Codewars,"{'totalAuthored': 1, 'totalCompleted': 135}",846,9985,Megg Gawat,"{'overall': {'rank': -5, 'name': '5 kyu', 'col...","[python, java, javascript, c#]",megawatt
9,MSU,"{'totalAuthored': 0, 'totalCompleted': 114}",1527,3791,Andrey,"{'overall': {'rank': -3, 'name': '3 kyu', 'col...",None,Dront


In [58]:
def flatten(data, col_list):
    for column in col_list:
        flattened = pd.DataFrame(dict(data[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + colname for colname in columns]
        data = pd.concat([data, flattened], axis=1)
        data = data.drop(column, axis=1)
    return data

In [ ]:
nested_columns = ['codeChallenges', 'ranks']

flat = flatten(data, nested_columns)

In [60]:
nested_columns = ['ranks_languages', 'ranks_overall']
flat = flatten(flat, nested_columns)

flat.head(10)

,clan,honor,leaderboardPosition,name,skills,username,codeChallenges_totalAuthored,codeChallenges_totalCompleted,ranks_languages_c,ranks_languages_coffeescript,...,ranks_languages_java,ranks_languages_javascript,ranks_languages_python,ranks_languages_ruby,ranks_languages_rust,ranks_languages_sql,ranks_overall_color,ranks_overall_name,ranks_overall_rank,ranks_overall_score
0,The Software Guild,11,340049,None,None,c.e.buco,0,5,NaN,NaN,...,"{'rank': -8, 'name': '8 kyu', 'color': 'white'...",NaN,NaN,NaN,NaN,NaN,white,8 kyu,-8,12
1,IIT BHU,16,298013,None,None,Tciitb,0,5,NaN,NaN,...,NaN,NaN,"{'rank': -7, 'name': '7 kyu', 'color': 'white'...",NaN,NaN,NaN,white,7 kyu,-7,24
2,MITx,501,22996,None,None,Andrew_8,0,58,NaN,NaN,...,NaN,"{'rank': -6, 'name': '6 kyu', 'color': 'yellow...","{'rank': -5, 'name': '5 kyu', 'color': 'yellow...",NaN,NaN,NaN,yellow,5 kyu,-5,582
3,GTS RO,297,48226,,[],mi5hu,0,49,NaN,NaN,...,NaN,NaN,NaN,"{'rank': -5, 'name': '5 kyu', 'color': 'yellow...",NaN,NaN,yellow,5 kyu,-5,387
4,IIT BHU,188,74276,None,None,abhishek0318,0,21,NaN,NaN,...,NaN,NaN,"{'rank': -6, 'name': '6 kyu', 'color': 'yellow...",NaN,NaN,NaN,yellow,6 kyu,-6,228
5,Central Philippine University,208,67793,Kenneth Ken L. Tan,[],ken11zer01,0,39,NaN,NaN,...,NaN,"{'rank': -6, 'name': '6 kyu', 'color': 'yellow...",NaN,NaN,NaN,"{'rank': -8, 'name': '8 kyu', 'color': 'white'...",yellow,6 kyu,-6,201
6,None,1206,5662,Sergey,[],hardentoo,1,115,"{'rank': -6, 'name': '6 kyu', 'color': 'yellow...",NaN,...,NaN,NaN,"{'rank': -6, 'name': '6 kyu', 'color': 'yellow...",NaN,NaN,NaN,blue,4 kyu,-4,1637
7,Government Drones,5585,445,Jason Levine,"[java, javascript, html, css, sql]",delfuego,1,251,NaN,"{'rank': -4, 'name': '4 kyu', 'color': 'blue',...",...,NaN,"{'rank': -2, 'name': '2 kyu', 'color': 'purple...",NaN,"{'rank': -6, 'name': '6 kyu', 'color': 'yellow...",NaN,NaN,purple,2 kyu,-2,9426
8,Codewars,846,9985,Megg Gawat,"[python, java, javascript, c#]",megawatt,1,135,NaN,NaN,...,"{'rank': -6, 'name': '6 kyu', 'color': 'yellow...","{'rank': -7, 'name': '7 kyu', 'color': 'white'...","{'rank': -5, 'name': '5 kyu', 'color': 'yellow...","{'rank': -8, 'name': '8 kyu', 'color': 'white'...",NaN,NaN,yellow,5 kyu,-5,546
9,MSU,1527,3791,Andrey,None,Dront,0,114,NaN,NaN,...,NaN,NaN,"{'rank': -3, 'name': '3 kyu', 'color': 'blue',...",NaN,"{'rank': -8, 'name': '8 kyu', 'color': 'white'...",NaN,blue,3 kyu,-3,2249


In [98]:
def get_languages(url='https://www.codewars.com/kata/latest/my-languages', ignore={'All'}): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text.lower() for e in soup.select('#language_filter option')}.difference(ignore) 

languages = get_languages()
languages

{'all',
 'bf (beta)',
 'c',
 'c#',
 'c++',
 'clojure',
 'coffeescript',
 'crystal',
 'dart',
 'elixir',
 'elm (beta)',
 'erlang (beta)',
 'f#',
 'fortran (beta)',
 'go',
 'groovy (beta)',
 'haskell',
 'java',
 'javascript',
 'julia (beta)',
 'kotlin (beta)',
 'lua (beta)',
 'my languages',
 'nasm (beta)',
 'nim (beta)',
 'objective-c (beta)',
 'ocaml (beta)',
 'php',
 'powershell (beta)',
 'purescript (beta)',
 'python',
 'r (beta)',
 'ruby',
 'rust',
 'scala (beta)',
 'shell',
 'solidity (beta)',
 'sql',
 'swift',
 'typescript'}

In [99]:
languages = get_languages()
user_columns = ['username', 'honor', 'member since', 'github', 'linkedin', 'clan', 'skills', 
                'challenges_authored', 'challenges_completed']

In [100]:
columns = [u for u in user_columns] + [l for l in languages]
df_users = pd.DataFrame(columns = columns)
df_users.head()

,username,honor,member since,github,linkedin,clan,skills,challenges_authored,challenges_completed,r (beta),...,elixir,nim (beta),c,crystal,erlang (beta),objective-c (beta),ocaml (beta),ruby,elm (beta),dart


In [94]:
df_users.columns

Index(['username', 'honor', 'Member_Since', 'github', 'linkedin', 'clan',
       'skills', 'codeChallenges_totalAuthored',
       'codeChallenges_totalCompleted', 'Shell', 'Nim (Beta)', 'Dart',
       'Fortran (Beta)', 'Clojure', 'Objective-C (Beta)', 'My Languages',
       'Julia (Beta)', 'SQL', 'Lua (Beta)', 'Haskell', 'PureScript (Beta)',
       'Swift', 'Erlang (Beta)', 'Java', 'R (Beta)', 'Go', 'C#', 'Python',
       'Scala (Beta)', 'Solidity (Beta)', 'NASM (Beta)', 'C++', 'Elm (Beta)',
       'TypeScript', 'CoffeeScript', 'JavaScript', 'PHP', 'Groovy (Beta)', 'C',
       'Kotlin (Beta)', 'Ruby', 'Elixir', 'PowerShell (Beta)', 'BF (Beta)',
       'Crystal', 'OCaml (Beta)', 'F#', 'Rust'],
      dtype='object')

In [107]:
total_score = alberto['ranks']['overall']['score']
total_score

888

In [109]:
skills = alberto['skills']
skills

['java', 'c', 'python', 'jquery', 'javascript', 'crm', 'erp', 'itil']

10

In [119]:
alberto

{'username': 'albertogcmr',
 'name': 'Alberto García cobo',
 'honor': 727,
 'clan': 'Ironhack',
 'leaderboardPosition': 12890,
 'skills': ['java',
  'c',
  'python',
  'jquery',
  'javascript',
  'crm',
  'erp',
  'itil'],
 'ranks': {'overall': {'rank': -4,
   'name': '4 kyu',
   'color': 'blue',
   'score': 888},
  'languages': {'python': {'rank': -4,
    'name': '4 kyu',
    'color': 'blue',
    'score': 857},
   'sql': {'rank': -7, 'name': '7 kyu', 'color': 'white', 'score': 24},
   'c': {'rank': -8, 'name': '8 kyu', 'color': 'white', 'score': 10}}},
 'codeChallenges': {'totalAuthored': 0, 'totalCompleted': 89}}

In [122]:
df_users.head()

,username,honor,member since,github,linkedin,clan,skills,challenges_authored,challenges_completed,r (beta),...,elixir,nim (beta),c,crystal,erlang (beta),objective-c (beta),ocaml (beta),ruby,elm (beta),dart


In [139]:
def get_value_from_json(user_json, value='username'): 
    ''' 
    value en nivel 1 del json: 
    possible values = ['username', 'honor', 'clan', 'leaderboardPosition', 'skills']
    
    '''
    return user_json[value]

def get_score_language(user_json, language): 
    try: 
        score = user_json['ranks']['languages'][language]['score']
    except: 
        score = 0
    else: 
        pass
    finally: 
        pass
    return score

get_score_language(user_json=alberto, language='sca')
# get_value_from_json(alberto, 'skills')

0

In [130]:
def create_row(data, new_user): 
    res = data
    res = res.append(other={'username': 'asd'}, ignore_index=True)
    return res


for lang in languages: 
    # print(get_score_language(user_json=alberto, language=lang))
    pass
    
df_users = create_row(data=df_users, new_user=alberto)
df_users.head()

,username,honor,member since,github,linkedin,clan,skills,challenges_authored,challenges_completed,r (beta),...,elixir,nim (beta),c,crystal,erlang (beta),objective-c (beta),ocaml (beta),ruby,elm (beta),dart
0,asd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,asd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
